In [1]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
import os
print(os.getcwd())
# 1. Read datasets
full_df = pd.read_csv("data/full_df_cleaned.csv")


/Users/jadevega/CIS4930-ML-Final-Project


In [2]:
target_column = [
    'tempo',
    'valence', 
    'energy']


X = full_df[target_column].values


knn_model = NearestNeighbors(
    metric='cosine',  
    algorithm='brute', 
    n_neighbors=200
)

knn_model.fit(X)
print("KNN model fitted successfully.")

KNN model fitted successfully.


In [3]:
from collections import Counter 
def recommend_song(song_list, n_recs=10): 
    seed_indices = [] 
    for track_name, artist_name in song_list:
        t = track_name.lower()
        a = artist_name.lower()

        mask = (
            full_df["track_name"].str.lower() == t
        ) & (
            full_df["track_artist"].str.lower() == a
        )

        if mask.sum() > 0:
            idx = full_df[mask].index[0]
            seed_indices.append(idx)
        else:
            print(f"Song not found: {track_name} — {artist_name}")

    if len(seed_indices) == 0:
        return "No valid input songs found in dataset."
    for i, idx in enumerate(seed_indices):
        print(f"\nSeed {i+1}: index = {idx}")
        print(full_df.loc[idx, ["track_name", "track_artist", "mood"]])
    seed_moods = [full_df.loc[idx, "mood"] for idx in seed_indices]
    mood_target = Counter(seed_moods).most_common(1)[0][0]

    print("\nDetected seed moods:", seed_moods)
    print("Chosen mood_target →", mood_target)
    
    query_vector = X[seed_indices].mean(axis=0).reshape(1, -1) 
    distances, neighbors = knn_model.kneighbors( query_vector, n_neighbors=n_recs + len(seed_indices) + 20 ) 
    neighbors = neighbors[0] 
    
    filtered = [idx for idx in neighbors if idx not in seed_indices] 
    unique = [] 
    seen = set() 

    for idx in filtered: 
        if full_df.loc[idx, "mood"] != mood_target: 
            continue 
        song_id = ( 
            full_df.loc[idx, "track_artist"], 
            full_df.loc[idx, "track_name"] ) 
        if song_id not in seen: 
            seen.add(song_id) 
            unique.append(idx) 
        if len(unique) == n_recs: 
            break 
    return full_df.iloc[unique].reset_index(drop=True), unique

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
def cosine_similarity_score(song_list, rec_indices):
    seed_indices = []

    for track_name, artist_name in song_list:
        t = track_name.lower()
        a = artist_name.lower()

        mask = (
            full_df["track_name"].str.lower() == t
        ) & (
            full_df["track_artist"].str.lower() == a
        )

        if mask.sum() > 0:
            idx = full_df[mask].index[0]
            seed_indices.append(idx)

    query_vector = X[seed_indices].mean(axis=0).reshape(1, -1)
    rec_vectors = X[rec_indices]

    scores = cosine_similarity(query_vector, rec_vectors)

    return scores.flatten()

songs_sad = [
    ("Here With Me", "Marshmello"),
    ("BOSS", "NCT"), 
    ("Choir - Remix", "Guy Sebastian"), 
    ("Somebody I'm Not", "Davai")
]

songs_happy= [
    ("Thorns", "Faustix"), 
    ("Forever Young", "John De Sohn"),         
    ("HIP", "Mamamoo"),
    ("Turning Up", "ARASHI")
        ]

songs_chill = [
    ("All the Time", "Zara Larsson"), 
    ("SOS", "Avicii"), 
    ("Stay", "Zedd"), 
    ("Replay", "Davai")
]

songs_hype = [
    ("With You", "Kaskade"), 
    ("Giulia", "Prince Paris"), 
    ("All This Time", "Deorro"), 
    ("Penny", "Dimitri Vangelis & Wyman")
]
recommended_chill, rec_index_chill = recommend_song(songs_chill, n_recs = 10)
print("Chill Recommendations")
print(recommended_chill)

recommended_sad, rec_idx_sad = recommend_song(songs_sad, n_recs=10)
print("Sad Recommendations")
print(recommended_sad)

recommended_happy, rec_index_happy = recommend_song(songs_happy, n_recs=10)
print("Happy Recommendations")
print(recommended_happy)

recommended_hype, rec_index_hype = recommend_song(songs_hype, n_recs=10)
print("Hype Recommendations")
print(recommended_hype)

similarity_scores_chill = cosine_similarity_score(songs_chill, rec_index_chill)
print("\nCosine Similarity Scores for Chill Songs:")
print(similarity_scores_chill)

similarity_scores_sad = cosine_similarity_score(songs_sad, rec_idx_sad)
print("\nCosine Similarity Scores for Sad Songs:")
print(similarity_scores_sad)

similarity_scores = cosine_similarity_score(songs_happy, rec_index_happy)
print("\nCosine Similarity Scores for Happy Songs:")
print(similarity_scores)

similarity_scores_hype = cosine_similarity_score(songs_hype, rec_index_hype)
print("\nCosine Similarity Scores for Happy Songs:")
print(similarity_scores_hype)





Seed 1: index = 80
track_name      All the Time
track_artist    Zara Larsson
mood                   chill
Name: 80, dtype: object

Seed 2: index = 93
track_name         SOS
track_artist    Avicii
mood             chill
Name: 93, dtype: object

Seed 3: index = 130
track_name       Stay
track_artist     Zedd
mood            chill
Name: 130, dtype: object

Seed 4: index = 307
track_name      Replay
track_artist     Davai
mood             chill
Name: 307, dtype: object

Detected seed moods: ['chill', 'chill', 'chill', 'chill']
Chosen mood_target → chill
Chill Recommendations
      track_artist                                   track_name  \
0  Porter Robinson                      Divinity - filous Remix   
1     Junko Ohashi                                  カナディアン・ララバイ   
2            Diplo                 Lonely (with Jonas Brothers)   
3    Play-N-Skillz                       Si Una Vez - If I Once   
4              ZHU                 Working For It - Bonus Track   
5              KKG 